# Reconstruction of band using Markov Random Field Model

## Model setup

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt

from fuller.mrfRec import MrfRec
from fuller.utils import loadHDF

import os

In [ ]:
data_path = '../' # Put in Path to a storage of at least 20 Gbyte free space.
if not os.path.exists(data_path + "/data.zip"):
    os.system(f"curl -L --output {data_path}/data.zip https://zenodo.org/records/7314278/files/data.zip")
if not os.path.isdir(data_path + "/data"):
    os.system(f"unzip -d {data_path} -o {data_path}/data")

In [ ]:
# Load preprocessed data
data = loadHDF('../data/pes/1_sym.h5')
E = data['E']
kx = data['kx']
ky = data['ky']
I = data['V']

In [ ]:
# Create MRF model
mrf = MrfRec(E=E, kx=kx, ky=ky, I=I, eta=.12)
mrf.I_normalized = True

## Initialization

In [ ]:
# Initialize mrf model with band structure approximation from DFT
path_dft = '../data/theory/WSe2_PBEsol_bands.mat'

band_index = 4
offset = .5
k_scale = 1.1

kx_dft, ky_dft, E_dft = mrf.loadBandsMat(path_dft)
mrf.initializeBand(kx=kx_dft, ky=ky_dft, Eb=E_dft[band_index,...], offset=offset, kScale=k_scale, flipKAxes=True)

In [ ]:
# Plot slices with initialiation to check offset and scale
mrf.plotI(ky=0, plotBandInit=True, cmapName='coolwarm')
mrf.plotI(kx=0, plotBandInit=True, cmapName='coolwarm')

## Reconstruction

In [ ]:
# Run optimization to perform reconstruction
eta = .1
n_epochs = 150

mrf.eta = eta
mrf.iter_para(n_epochs)

In [ ]:
# Plot results
mrf.plotBands()
mrf.plotI(ky=0, plotBand=True, plotBandInit=True, plotSliceInBand=False, cmapName='coolwarm')
mrf.plotI(ky=0.5, plotBand=True, plotBandInit=True, plotSliceInBand=False, cmapName='coolwarm')
mrf.plotI(kx=0, plotBand=True, plotBandInit=True, plotSliceInBand=False, cmapName='coolwarm')
mrf.plotI(kx=0.5, plotBand=True, plotBandInit=True, plotSliceInBand=False, cmapName='coolwarm')

In [ ]:
# Save results
path_save = 'reconstructed_bands'
mrf.saveBand(path_save + 'mrf_rec_%02i.h5' % band_index, index=band_index)